In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
# import get_state as getstate
# import utility as util
# import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(ANi,ACu,epCu,epNi,tpd,tpp,tz,pds,pdp,pps,ppp,Upp,\
                    d_double,p_double,U, S_val, Sz_val, AorB_sym):                                            #改
    if Norb==7:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)                                           #改
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta)
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
    tz_fac = ham.set_tz(Norb,tz)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    T_z    = ham.create_tz_matrix(VS,tz_fac)
    Esite  = ham.create_edep_diag_matrix(VS,ANi,ACu,epCu,epNi)                                                        #改
    
    H0 =  T_pd+ T_pp + T_z + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_Ni_interaction_matrix_ALL_syms(VS,d_double,p_double,S_val, Sz_val,AorB_sym,ANi,Upp)
        Hint2 = ham.create_Cu_interaction_matrix_ALL_syms(VS,d_double,p_double,S_val, Sz_val,AorB_sym,ACu,Upp)
        
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint +Hint2 
        else:
            H = H0 + Hint + Hint2 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
#             if Norb==9:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,ep,tpd,vals[0])
#                 util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,ep,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
#             elif Norb==10 or Norb==11 or Norb==12:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,ep,pds,pdp,vals[0])
#                 util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,ep,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
#         '''
#         Compute A(w) for various states
#         '''
#         # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)

#         # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)

#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    edNi   = pam.edNi
    edCu   = pam.edCu

    ANis = pam.ANis
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double, p_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        for a in range(0,len(pam.tzs)):
            tz = pam.tzs[a] 
            for tpd in pam.tpds:
                for epCu in pam.epCus:
                    for epNi in pam.epNis:               
                        for ANi in pam.ANis:
                            for ACu in pam.ACus:
    #                            util.get_atomic_d8_energy(ANi,B,C)
                                for tpp in pam.tpps:
                                    for Upp in pam.Upps:
                                        print ('===================================================')                     #gai
                                        print ('ANi=',ANi, 'ACu=',ACu,'epCu=', epCu, 'epNi=',epNi,' tpd=',tpd,' tpp=',tpp,' Upp=',Upp )
                                        compute_Aw_main(ANi,ACu,epCu,epNi,tpd,tpp,tz,0,0,0,0,Upp,\
                                                        d_double,p_double,U, S_val, Sz_val, AorB_sym)
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for a in range(0,len(pam.tzs)):
            tz = pam.tzs[a]                                                                                                #修改
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pdp = pam.pdps[ii]
                for epCu in pam.epCus:
                    for epNi in pam.epNis:
                        for ANi in pam.ANis: 
                            for ACu in pam.ACus:
    #                         util.get_atomic_d8_energy(ANi,B,C)
                                for Upp in pam.Upps:
                                    print ('===================================================')
                                    print ('ANi=',ANi, 'ACu=',ACu,'epCu=',epCu, 'epNi=',epNi,' pds=',pds,' pdp=',pdp,' pps=',pps,' ppp=',ppp,' Upp=',Upp,'tz=',tz)
                                    compute_Aw_main(ANi,ACu,epCu,epNi,0,0,tz,pds,pdp,pps,ppp,Upp,\
                                                    d_double,p_double,U, S_val, Sz_val, AorB_sym)   ,


                        
    print("--- %s seconds ---" % (time.time() - start_time))

Populating the interactive namespace from numpy and matplotlib
Ni_Cu_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px1', 'py1', 'pz1']
O2_orbs =  ['px2', 'py2', 'pz2']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 5
VS.dim =  51076
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 25887 25189 0
len(d_list) 50
len(p_list) 648
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.8787557  -2.8787557  -2.22082369 -2.15532581 -2.15532581 -2.08879562
 -2.08879562 -2.00133512 -2.00133512 -2.00133512]
eigenvalue =  -2.8787556963561256
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 1 dx2y2 0 0 0 S= 0 Sz= 0 , we

sumweight= 0.9999999999999778
sumjian= 0.12788225618138804
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 1.5
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-4.72584498 -3.91006287 -3.69080457 -3.40848252 -3.3209989  -2.96855537
 -2.45029097 -2.45029097 -2.39813042 -2.39813042]
eigenvalue =  -4.725844976329599
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.0040314556334198415
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.004127912403885604
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0041279124038855735
 state  py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.004031433995275544
 state  px1 -1 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.024298746049758357
 state  px1 -1

sumweight= 0.9999999999999925
sumjian= 0.148956412586474
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 2.5
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-6.43018203 -5.8078151  -5.51094951 -5.05534052 -4.9300079  -4.15306957
 -4.11240672 -4.11240672 -4.09661883 -4.0014151 ]
eigenvalue =  -6.430182030214731
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.003696711109733443
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.003817721365721867
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0038177213657218877
 state  py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.003696702082289492
 state  px1 -1 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.02431179595477174
 state  px1 -1 0 0

sumweight= 0.9999999999999855
sumjian= 0.15727436121645405
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 3.5
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-8.24570358 -7.82634817 -7.48376354 -6.85194198 -6.70578582 -6.13243451
 -6.12581365 -5.92130526 -5.92130526 -5.81031363]
eigenvalue =  -8.245703584876164
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.0033611239064516283
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.003507949440018993
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0035079494400190076
 state  py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0033611201626054916
 state  px1 -1 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.02352039855644195
 state  px1 -1

sumweight= 0.9999999999999943
sumjian= 0.16201480622250403
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 4.5
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-10.12186146  -9.90288696  -9.52978508  -8.71890457  -8.55978318
  -8.25651813  -8.18345925  -7.8014643   -7.8014643   -7.69047267]
eigenvalue =  -10.12186146098171
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.0030950354611970618
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.003273539385525766
 state  px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0032735393855258164
 state  py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0030950338714656467
 state  px1 -1 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.022717076574041088
 st

...

sumweight= 1.0000000000000038
sumjian= 0.009540096796254316
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 6.5
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-14.14208209 -13.97190976 -13.73052661 -12.66800935 -12.55558995
 -12.38979408 -12.38151828 -11.88832376 -11.88832376 -11.87856095]
eigenvalue =  -14.142082091690874
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.01590094979517909
 state  px1 -1 0 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.015900949795177928
 state  px1 -1 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.009579844250846212
 state  px1 -1 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.00957984425084546
 state  px1 -1 0 0 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.0160103931102310

 state  py2 0 -1 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.01569075982451798
 state  py2 0 -1 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.015690759824515812
 state  py2 0 -1 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.009995878618554587
 state  py2 0 -1 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.009995878618553217
 state  py2 0 -1 0 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.015757194213034256
 state  py2 0 -1 0 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.01575719421303209
 state  py2 0 -1 1 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.00944327725897744
 state  py2 0 -1 1 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.009443277258976122
 state  dx2y2 0 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.07057533240341
 state  dx2y2 0 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.07057533240340143
 state  d3z2r2 0 0 0 dx2y2 0 0 0 S= 1 Sz= 0 , weight =  0.12436534433977356
 state  d3z2r2 0 0 1 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.07676147302857073
 state  d3z2r2 0 0 1 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.07676147302857979
 state  d3z2r2 0 0

sumweight= 0.9999999999999685
sumjian= 0.007513051097851124
ANi= 10.0 ACu= 6.0 epCu= 3.0 epNi= 7.0  pds= 1.501110699893027  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0 tz= 9.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
start create_interaction_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-19.51842388 -19.07898619 -18.89991118 -18.29875593 -17.78931176
 -17.78931176 -17.74053159 -17.68423017 -17.68423017 -17.68386397]
eigenvalue =  -19.518423879497156
Compute the weights in GS (lowest Aw peak)
 state  px1 -1 0 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.015453956000158616
 state  px1 -1 0 0 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.01545395600015747
 state  px1 -1 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.010497966043845909
 state  px1 -1 0 1 d3z2r2 0 0 0 S= 0 Sz= 0 , weight =  0.010497966043845187
 state  px1 -1 0 0 d3z2r2 0 0 1 S= 0 Sz= 0 , weight =  0.015487528133427

sumweight= 0.999999999999982
sumjian= 0.006651001684864336
--- 386.9886841773987 seconds ---


<Figure size 432x288 with 0 Axes>